In [ ]:
import numpy as np
import torch
import json
import random
import sys
import unidecode 
from numpy import dot
from numpy.linalg import norm
from scipy.stats import kendalltau, spearmanr
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel

In [ ]:
lang='kn'

In [ ]:

def load_jsonl(file_path):
    res = []
    with open(file_path, encoding='utf-8') as dfile:
        for line in dfile.readlines():
            res.append(json.loads(line.strip()))
    return res


filename = "./plan_order/{lang}/{lang}_order.jsonl".format(lang=lang)

reader  = load_jsonl(filename)

In [ ]:
model_name = "bert-base-multilingual-cased"
model = AutoModel.from_pretrained(model_name,output_hidden_states=True, cache_dir="/scratch")
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir="/scratch")



In [ ]:

languages_map = {
    'en': {"label": "English", 'id': 0},
    'hi': {"label": "Hindi", 'id': 1},
    'te': {"label": "Telugu", 'id': 2},
    'bn': {"label": "Bengali", 'id': 3},
    'pa': {"label": "Punjabi", 'id': 4},
    'ur': {"label": "Urdu", 'id': 5},
    'or': {"label": "Odia", 'id': 6},
    'as': {"label": "Assamese", 'id': 7},
    'gu': {"label": "Gujarati", 'id': 8},
    'mr': {"label": "Marathi", 'id': 9},
    'kn': {"label": "Kannada", 'id': 10},
    'ta': {"label": "Tamil", 'id': 11},
    'ml': {"label": "Malayalam", 'id': 12}
}


def get_nodes(n):
    n = n.strip()
    n = n.replace('(', '')
    n = n.replace('\"', '')
    n = n.replace(')', '')
    n = n.replace(',', ' ')
    n = n.replace('_', ' ')
    n = unidecode.unidecode(n)
    return n


def get_relation(n):
    n = n.replace('(', '')
    n = n.replace(')', '')
    n = n.strip()
    n = n.split()
    n = "_".join(n)
    return n


def linear_fact_str(fact, enable_qualifiers=True):
    fact_str = ['<R>', get_relation(fact[0]).lower(), '<T>', get_nodes(fact[1]).lower()]
    qualifier_str = [' '.join(['<QR>', get_relation(x[0]).lower(), '<QT>', get_nodes(x[1]).lower()]) for x in fact[2]]
    if enable_qualifiers and len(fact[2])>0:
        fact_str += [' '.join(qualifier_str)]
    return fact_str

def process_facts(facts, entity, section, language):
    """ linearizes the facts on the encoder side """
    linearized_facts = []
    for i in range(len(facts)):
        linearized_facts += linear_fact_str(facts[i], enable_qualifiers=True)
    processed_facts_str = ' '.join(linearized_facts)
    pre_string = "<H> %s %s <S> %s" % (entity.lower(), processed_facts_str, section)
#     pre_string = "<H> %s %s " % (entity.lower(), processed_facts_str)
    return "generate %s : %s" % (languages_map[language]['label'].lower(), pre_string)



In [ ]:
def get_word_idx(sent: str, word: str):
    return sent.split(" ").index(word)
  
def get_hidden_states(sent, tokenizer, model, layers):
    """Push input IDs through model. Stack and sum `layers` (last four by default).
    Select only those subword token outputs that belong to our word of interest
    and average them."""

    encoded = tokenizer.encode_plus(sent, return_tensors="pt")
    with torch.no_grad():
        output = model(**encoded)

    # Get all hidden states
    # print (output.cross_attentions)
    states = output.hidden_states
    # Stack and sum all requested layers
    output = torch.stack([states[i] for i in layers]).sum(0).squeeze()

    wordlist = sent.split(' ')
    finvec = []
    for idx in range(len(wordlist)) :
        token_ids_word = np.where(np.array(encoded.word_ids()) == idx)
        #print(token_ids_word,"===")
        word_tokens_output = output[token_ids_word]
        #print (word_tokens_output)
        finvec.append(word_tokens_output.mean(dim=0))

    return finvec

In [ ]:
# Use last four layers by default
layers = [-4, -3, -2, -1] 


def getscore (sentence, factword):
    currsim  = -1
    currind = -1
    count = 0
    for aword in sentence :
        cosval = dot(aword, factword)/(norm(aword)*norm(factword))
        #print (cosval, " this is cosine similarity")
        if cosval > currsim :
            currsim = cosval
            currind = count 
        count += 1
    return currsim, currind 

from tqdm import tqdm

ref_order = []
pred_order = []

for obj in tqdm(reader) :
    #print (obj)
    sent  = obj['sentence']
    factlist = obj['facts']
    ref_order.append(obj['order'])
    
    sent_vec = get_hidden_states(sent, tokenizer, model, layers)
    #print (sent)
    #print (len(sent_vec))

    scorelist  = []
    

    for i in factlist :
        #print (i)
        outputs = get_hidden_states(i[1], tokenizer, model, layers)
        bestscore = -1
        currind = -1
        indcount = 0
        #print (len(outputs))
        for fact_word in outputs:
            #print ("checking for word ", i[1].split()[indcount])
            indcount += 1
            score,ind = getscore(sent_vec,fact_word)
            #print ("best score is ", score , " with word " ,sent.split()[ind])
            if score > bestscore:
                bestscore = score 
                currind = ind 

        scorelist.append(currind)

    scorelist = np.array(scorelist)
    final_ordering =  np.argsort(scorelist)
    
    # final ordering is an array where each element depicts the index of the fact (from the original fact list) which should appear at that index in the sorted order 
    # for example an array [3,1,2,0] represents that the first fact in sorted order should be the one having index 3 in original fact list, second fact is the one at index 1 in the original fact list ... similarly third was at index 2 and last at index 0. 
    pred_order.append(final_ordering.tolist())

In [ ]:
qidx = random.choice([z for z in range(100)])
print(pred_order[qidx])
print(ref_order[qidx])

In [ ]:
def get_accuracy(a, b):
    assert len(a)==len(b), "unequal size of list"
    count = 0
    for y, z in zip(a, b):
        if y==z:
            count+=1
    return count/len(a)

In [ ]:
avg_spearman = np.array([spearmanr(y, z)[0] for y, z in zip(pred_order, ref_order)])
avg_kendalltau = np.array([kendalltau(y, z)[0] for y, z in zip(pred_order, ref_order)])

print("spearman", avg_spearman.mean())
print("kendalltau", avg_kendalltau.mean())
print("accuracy", get_accuracy(ref_order, pred_order))